In [5]:
import numpy as np
import numpy.ma as ma

import time

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt

In [16]:
%load_ext autoreload
%autoreload 1
%aimport sigproc
%aimport config_svm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IndentationError: expected an indented block (svm.py, line 31)

In [ ]:
%aimport svm

In [ ]:
change_printopt = config_svm.printoptions

In [7]:
pic_dir = config_svm.pic_dir

NameError: name 'config_svm' is not defined

In [8]:
import sys
Eps =  config_svm.Eps #sys.float_info.epsilon

NameError: name 'config_svm' is not defined

# 3. Эксперименты на реальных данных

Создаем многомерный массив, размерность (количетво нужных символов (английские буквы)= 26x2,
                                         количество респондентов=60,
                                         количество повторений=2,
                                         вектор измерений (x, y) = 2,
                                         максимальное количество измерений = количество измерений) //надо посчитать

In [9]:
import re
from os import walk

r_strokenum = re.compile('\s+NUMSTROKES (\d)')
r_pntnum = re.compile('\s+POINTS (\d+) #*')
r_word = re.compile('WORD ([a-zA-Z]) (trn|tst)_(UJI|UPV)_W(\d\d)-(01|02)', re.A)
strokenum = 0
num = 0
cnt = 0
max_d = 0

file_name = 'data/uji-penchars/ver2/ujipenchars2.txt'
data_file = open(file_name)
text_data = data_file.read().splitlines()
it_text = iter(text_data)

for line in it_text:
    if r_word.match(line):
        letter = r_word.match(line).group(1)
        num = 0
        line = next(it_text)
        strokenum = int(r_strokenum.match(line).group(1))
        cnt += 1
        while(strokenum):
            line = next(it_text)
            num += int(r_pntnum.match(line).group(1))
            strokenum -= 1 
        if max_d < num:
            max_d = num
            #print(letter, max_d) - Напечатать буквы макимизирующие вырпжение

In [10]:
keys = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
 "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
 "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M",
 "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
values = list(range(len(keys)))
lexicon = dict(zip(keys, values))
anti_lexicon = dict(zip(values, keys))
frame_size = np.array([13.6, 20.4]) 
UJI_ratio, UPV_ratio = 100, 152

print("Больше всего точек буквы = ", max_d)
masked_fill_value = 1.e-20
real_data = masked_fill_value*np.ones((len(keys), 60, 2, 2, max_d), dtype=np.float64)

Больше всего точек буквы =  392


In [11]:
print("Mbytes in data stored as masked arrays with length {0:d}: {1:.5f} Mb".format(max_d, real_data.nbytes / 2**(20)))

Mbytes in data stored as masked arrays with length 392: 37.32422 Mb


In [12]:
r_word = re.compile('WORD ([a-zA-Z]) (trn|tst)_(UJI|UPV)_W(\d\d)-(01|02)', re.A)
r_pntnum = re.compile('\s+POINTS (\d+) #((\s+-?\d+)*)')

it_text = iter(text_data)
for line in it_text:
    if r_word.match(line):
        letter, UnivDivider, writer, repet = r_word.match(line).group(1, 3, 4, 5)
        line = next(it_text)
        strokenum = int(r_strokenum.match(line).group(1))
        stroke_offset = 0
        while(strokenum):
            line = next(it_text)
            pntnum_m = r_pntnum.match(line)
            pnts_num = int(pntnum_m.group(1))
            #print(pnts_num)
            #print(pntnum_m.group(2)[1:].split(sep=' '))
            pnts = np.array([int(pnt_xy) for pnt_xy in pntnum_m.group(2)[1:].split(sep=' ')])
            divider = 1
###can change divider parametr
            if UnivDivider == 'UPV':
                divider = UPV_ratio/UJI_ratio 
            #print(lexicon[letter], int(writer) - 1, int(repet) - 1)
            #try:
            real_data[lexicon[letter], int(writer) - 1, int(repet) - 1, 0,
                      stroke_offset:stroke_offset + pnts_num] = pnts[::2]/divider # x coords
            real_data[lexicon[letter], int(writer) - 1, int(repet) - 1, 1,
                      stroke_offset:stroke_offset + pnts_num] = pnts[1::2]/divider # y coords
            #except ValueError:
            #    print(pnts.shape)
            stroke_offset += pnts_num
            strokenum -= 1

In [13]:
real_data = ma.masked_values(real_data, value=masked_fill_value)

In [14]:
with config_svm.printoptions(suppress=True):
    print(real_data)

NameError: name 'config_svm' is not defined

In [ ]:
def vis_letters(letter_index, real_data,frame=frame_size, ratio=UJI_ratio, vert_lines=np.array([7.5,12.7]), verbose=True):
    points_xy = real_data[letter_index].compressed().reshape(2, -1)
    fig = plt.figure(figsize=frame/2)
    ax = fig.gca()
    ax.plot(points_xy[0], points_xy[1], 'k-', linewidth=0.5)
    ax.scatter(points_xy[0, 1:-1], points_xy[1, 1:-1], c='b', marker='o', s=30)
    ax.scatter(points_xy[0, 0], points_xy[1, 0], c=(0, 0.95, 0), marker='o', s=40) #start
    ax.scatter(points_xy[0, -1], points_xy[1, -1], c=(0.95, 0, 0), marker='o', s=40) #end
    xlim = [0.0, frame[0]*ratio]
    ylim = [0.0, frame[1]*ratio]
    ax.xaxis.tick_top()
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.plot(xlim, vert_lines[[0, 0]]*ratio, 'k', lw=1)
    ax.plot(ylim, vert_lines[[1, 1]]*ratio, 'k', lw=1)
    ax.set_xticks(np.arange(*(xlim + [100])))
    ax.set_yticks(np.arange(*(ylim + [100])))
    # We change the fontsize of minor ticks label 
    ax.tick_params(axis='both', which='major', labelsize=10)
    if verbose:
        ax.set_title("Буква '{0:s}'\tреспондент {1:d}\tпопытка {2:d}".format(
                anti_lexicon[letter_index[0]], letter_index[1] + 1, letter_index[2] + 1),fontsize=20, y=1.03)
    ax.invert_yaxis()
    ax.grid()     

In [15]:
for writer in range(1, 60, 10):
    for repet in range(2):
        vis_letters((lexicon['y'], writer, repet), real_data)

NameError: name 'vis_letters' is not defined

## Making DTW preprocessing

In [ ]:
def test(real_data, index_list):
    for ind in index_list:
        a = real_data[ind[0]].compressed().reshape(2, -1).T
        b = real_data[ind[1]].compressed().reshape(2, -1).T
        dist, D = sigproc.dtw(a, b, do_path=False)
        print("Буква '{0:s}'\tреспондент {1:d}\tпопытка 0 и 1: \t{2:.3f}".format(
                anti_lexicon[ind[0][0]], ind[0][1] + 1, dist))

In [ ]:
index_list = [[(lexicon['y'], wr-1, 0), (lexicon['y'], wr-1, 1)] for wr in [2, 22,32]]
test(real_data, index_list)

In [ ]:
X = real_data[[lexicon['f'], lexicon['e']], :11, 0].reshape(-1, 2, max_d)
y = np.ones((X.shape[0],1))
y[X.shape[0]/2:] *= -1

In [ ]:
SmSVM_feres_fe = svm.smooth_qp_primal_real_solver(X, y, np.array([0, 0, 0]), alpha=0, verbose=True)

In [ ]:
res_fe_al1 = svm.smooth_qp_primal_real_solver(X, y, np.array([0, 0, 0]), alpha=50, verbose=True)